# Deep Autoencoder com a base de dados MNIST

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense

In [2]:
(prev_train, _), (prev_test, _) = mnist.load_data()
prev_train.shape

prev_train = prev_train.astype("float32") / 255
prev_test = prev_test.astype("float32") / 255

### Usando o reshape para modificar a estrutura da base de dados

In [3]:
prev_train = prev_train.reshape((len(prev_train), np.prod(prev_train.shape[1:])))
prev_train.shape

(60000, 784)

In [4]:
prev_test = prev_test.reshape((len(prev_test), np.prod(prev_test.shape[1:])))
prev_test.shape

(10000, 784)

# Criando o modelo Deep Autoencoder

### vamos codificar os dados na sequencia
 784 -> 128 -> 64 -> 32 <- 64 <- 128 <- 784

    

In [5]:
autoencoder = Sequential()

## Criando o Encoder e o Decode

In [6]:
# Encode
autoencoder.add(Dense(units = 128, activation = "relu", input_dim = 784))
autoencoder.add(Dense(units = 64, activation = "relu"))
autoencoder.add(Dense(units = 32, activation = "relu"))

# Decode
autoencoder.add(Dense(units = 64, activation = "relu"))
autoencoder.add(Dense(units = 128, activation = "relu"))
autoencoder.add(Dense(units = 784, activation = "sigmoid"))

autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_5 (Dense)              (None, 784)               101136    
Total params: 222,384
Trainable params: 222,384
Non-trainable params: 0
__________________________________________________

## Compilando o modelo

In [7]:
autoencoder.compile(optimizer = "adam",
                   loss = "binary_crossentropy",
                   metrics = ["MeanAbsoluteError"])


## Treinamento do Deep Autoencoder

In [8]:
autoencoder.fit(prev_train,prev_train,
               epochs = 50, batch_size = 256,
               validation_data = (prev_test, prev_test))

Epoch 1/50
235/235 [==============================] - 7s 28ms/step - loss: 0.3368 - mean_absolute_error: 0.2043 - val_loss: 0.1650 - val_mean_absolute_error: 0.0848
Epoch 2/50
235/235 [==============================] - 5s 23ms/step - loss: 0.1573 - mean_absolute_error: 0.0791 - val_loss: 0.1378 - val_mean_absolute_error: 0.0656
Epoch 3/50
235/235 [==============================] - 5s 23ms/step - loss: 0.1359 - mean_absolute_error: 0.0642 - val_loss: 0.1248 - val_mean_absolute_error: 0.0568
Epoch 4/50
235/235 [==============================] - 6s 24ms/step - loss: 0.1242 - mean_absolute_error: 0.0561 - val_loss: 0.1169 - val_mean_absolute_error: 0.0510
Epoch 5/50
235/235 [==============================] - 6s 24ms/step - loss: 0.1171 - mean_absolute_error: 0.0511 - val_loss: 0.1116 - val_mean_absolute_error: 0.0476
Epoch 6/50
235/235 [==============================] - 6s 24ms/step - loss: 0.1117 - mean_absolute_error: 0.0474 - val_loss: 0.1078 - val_mean_absolute_error: 0.0447
Epoch 7/50

235/235 [==============================] - 5s 22ms/step - loss: 0.0837 - mean_absolute_error: 0.0274 - val_loss: 0.0833 - val_mean_absolute_error: 0.0271


# Criando o Codificador

In [9]:
dimensao_original = Input(shape = (784, ))
camada_encoder1 = autoencoder.layers[0]
camada_encoder2 = autoencoder.layers[1]
camada_encoder3 = autoencoder.layers[2]

encoder = Model(dimensao_original,
               camada_encoder3(camada_encoder2(camada_encoder1(dimensao_original))))
encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
Total params: 110,816
Trainable params: 110,816
Non-trainable params: 0
_________________________________________________________________


### Aplicando o encoder nas imagens de testes

In [10]:
imagens_codificadas = encoder.predict(prev_test)

In [11]:
imagens_codificadas.shape

(10000, 32)

In [14]:
imagens_decodificadas = autoencoder.predict(prev_test)

In [15]:
imagens_decodificadas.shape

(10000, 784)

# Visualizando as imagens

In [ ]:
num_imagens = 10

# Selecionando 10 imagens aleatórias da base de dados de teste
imagens_test = np.random.randint(prev_test.shape[0], size = num_imagens)

# Plotando as imagens
plt.figure(figsize = (18, 18))

for i, indice_img in enumerate(imagens_test):
    
    # Gráfico 01, plotar a imagem Original
    eixo = plt.subplot(10, 10, i + 1)
    plt.imshow(prev_test[indice_img].reshape(28, 28))
    plt.xticks(())
    plt.yticks(())
    
     # Gráfico 02, plotar a imagem codificada
    eixo = plt.subplot(10, 10, i + 1 + num_imagens)
    plt.imshow(imagens_code[indice_img].reshape(8, 4))
    plt.xticks(())
    plt.yticks(())
    
    # Gráfico 03, plotar a imagem Decodificada
    eixo = plt.subplot(10, 10, i + 1 + num_imagens * 2)
    plt.imshow(imagens_decode[indice_img].reshape(28, 28))
    plt.xticks(())
    plt.yticks(())